<a href="https://colab.research.google.com/github/bnelson05/Probing/blob/main/Probing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model and Data

# Pick a Pretrained Model
For example, distilbert-base-uncased-finetuned-sst-2-english or textattack/bert-base-uncased-SST-2.

Make sure you set it so it returns hidden states (e.g., output_hidden_states=True in the config/forward pass).

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.6 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.config.output_hidden_states = True

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

# Choose a Dataset
You can reuse an SST-2 dev/test split or a small subset of IMDB reviews—just keep it manageable (few hundred samples).

If you want to see domain differences, pick something that differs from the model’s training domain.

In [4]:
from datasets import load_dataset

imdb_ds = load_dataset("imdb")
train_subset = imdb_ds["train"].select(range(200))
test_subset = imdb_ds["test"].select(range(200))
print(train_subset[:3])

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'text': ['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far b

# Extract Hidden States

# Forward Pass
Tokenize each sentence, call model(**inputs), and ensure you’re receiving outputs.hidden_states.

# Layer Representation
For each sample, you’ll get a tuple of hidden states (like one for each layer).

Decide how to get a single vector for the sentence:

*   If it’s BERT, you can grab the [CLS] token’s embedding.
*   Or average the token embeddings if there’s no [CLS].

Save those embeddings (e.g., shape = [num_samples, hidden_dim]) for each layer, plus the true label.

In [ ]:
embeddings = []
true_labels = []

for i in range(len(train_subset["text"])):
  sentence = train_subset["text"][i]
  label = train_subset["label"][i]
  inputs = tokenizer(sentence, return_tensors="pt", truncation = True, max_length = 512)
  outputs = model(**inputs)
  hidden_states_received = outputs.hidden_states
  last_layer = hidden_states_received[-1]
  cls = last_layer[0, 0, :]
  embeddings.append(cls)
  true_labels.append(label)

# Training a Probe

# Logistic Regression
For each layer L, gather all embeddings into an (N, hidden_dim) array, with N = number of sentences.

Train a logistic regression (or SVM) to predict positive/negative.

# Evaluation
Use a held-out set or a small dev set to measure accuracy or F1.

# Compare Layers
You might pick layers 0, 3, 6, 9, 11—or each layer—then plot or table the results.

# Analysis & Write-Up

# Plot / Table
Show the classification performance (accuracy/F1) vs. layer index. Which layer is best?

# Discussion
Link it to your attention findings: if you saw strong attention to sentiment words in the 8th layer, do you also see that layer’s embeddings produce top performance in the probe?

Any surprises? Maybe the final layer is best, or maybe a middle layer outperforms it.